In [1]:
import numpy as np

from PIL import Image
from sklearn.datasets import fetch_california_housing

In [2]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader 
import matplotlib.pyplot as plt

In [3]:
from torchvision import transforms, datasets
import torch

In [4]:
import torch.nn.functional as F
import torch.nn as nn

In [5]:
#разделим на трейн и тест
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [6]:
X_train.shape, X_test.shape

((15480, 8), (5160, 8))

In [7]:
class MyOwnHousing(Dataset):
    def __init__(self, X, y):
      self.X = X
      self.y = y
    
    def __len__(self):
      return len(self.X)
    
    def __getitem__(self, idx):
      X = self.X.iloc[idx]
      y = self.y.iloc[idx]
      return torch.tensor(X, dtype = torch.float), torch.tensor(y, dtype = torch.float)

In [8]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.relu(x)
        if self.activation=="relu":
            return F.relu(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedForward, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, output_dim, "relu")
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.dp(x)
        x = self.fc2(x)
        return x

In [9]:
train_dataset = MyOwnHousing(X_train, y_train)
test_dataset = MyOwnHousing(X_test, y_test)

In [10]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=1)

In [11]:
test_loader = torch.utils.data.DataLoader(test_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [12]:
sample = next(iter(train_loader))
sample[0].shape, sample[1].shape

(torch.Size([128, 8]), torch.Size([128]))

### ADAM

In [13]:
net = FeedForward(8, 200, 1)

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [14]:
from tqdm import tqdm

In [15]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        x, y = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()        
        if i % 100 == 0:    # печатаем каждые 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,     1] loss: 0.054
[1,   101] loss: 1.861


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([120])) that is different to the input size (torch.Size([120, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
 10%|█         | 1/10 [00:02<00:25,  2.83s/it]

[2,     1] loss: 0.013
[2,   101] loss: 1.408


 20%|██        | 2/10 [00:05<00:19,  2.47s/it]

...

[9,     1] loss: 0.015
[9,   101] loss: 1.342


 90%|█████████ | 9/10 [00:20<00:02,  2.24s/it]

[10,     1] loss: 0.012
[10,   101] loss: 1.352


100%|██████████| 10/10 [00:22<00:00,  2.28s/it]

Training is finished!


### SGD

In [16]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [17]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        x, y = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 100 == 0:    # печатаем каждые 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.012
[1,   101] loss: 1.330


 10%|█         | 1/10 [00:02<00:19,  2.16s/it]

[2,     1] loss: 0.013
[2,   101] loss: 1.330


 20%|██        | 2/10 [00:04<00:17,  2.20s/it]

...

[9,     1] loss: 0.012
[9,   101] loss: 1.331


 90%|█████████ | 9/10 [00:19<00:02,  2.18s/it]

[10,     1] loss: 0.013
[10,   101] loss: 1.325


100%|██████████| 10/10 [00:21<00:00,  2.17s/it]

Training is finished!


### RMSProp

In [18]:
optimizer = torch.optim.RMSprop(net.parameters())

In [19]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        x, y = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 100 == 0:    # печатаем каждые 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.016
[1,   101] loss: 1.556


 10%|█         | 1/10 [00:03<00:29,  3.29s/it]

[2,     1] loss: 0.016
[2,   101] loss: 1.344


 20%|██        | 2/10 [00:05<00:21,  2.70s/it]

...

[9,     1] loss: 0.013
[9,   101] loss: 1.333


 90%|█████████ | 9/10 [00:21<00:02,  2.22s/it]

[10,     1] loss: 0.013
[10,   101] loss: 1.334


100%|██████████| 10/10 [00:23<00:00,  2.33s/it]

Training is finished!


RMSProp и SGD ведут себя примерно одинаково. При использовании Adam ошибка падает стабильнее.
